In [1]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [2]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [5]:
# Cell 3

wallet = KeyWallet.load("../../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../../keystores/balanced_test.json", key_data)
print(icon_service.get_balance(wallet.get_address())/10**18)
print(icon_service.get_balance(btest_wallet.get_address())/10**18)

760036469.3248185
16786078.8835182


In [6]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
760036469.3248185


In [7]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
16786078.8835182


In [8]:
user1 = KeyWallet.load("../../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

1034349.4737826
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
1012894.5146526
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [9]:
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx7c617e3fca4ba06b6ad203ce113245ae96a9d91e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx70cd5c86f0182d5ac0bd224562d929cd968d9132'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxeb91bc377b0620356787d9c9eb68152eb0c62d8a'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxfd7511ece084744154fed19dc34732681ad078e6'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9a3161c778eee2d5758371d3548c5599f76704ec'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx238cd1a1e3a9702d6c9c6dc130719472164db376'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx3825a86d52c5baf188ff29aa6a7fc2467e285885'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx140b49ea041457ebc4cd5e199f5723916bb50021'}}

In [12]:
# Cell 6
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    print(icon_service.call(call))
    return icon_service.call(call)


In [13]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print(get_scores_json(contracts))

../../core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/loans.zip
Status: 1

../../core_contracts/dividends.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/reserve.zip


...

Retrying in 1 seconds...
Status: 1

../../core_contracts/staking.zip
Status: 1

../../token_contracts/sicx.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

------------------------------------------------------------------------------------------------------------------
Calling setSicxAddress, with parameters {'_address': 'cx5e46cd5013a8f54a9b0d307d2995906cea3a5a2b'} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab', 'staking': 'cx96eba15c7a6814d2a10235b01e0b89845d7cb4bb', 'dividends': 'cx8c80bc90b668958a5603f19d41adb76c4f10a468', 'reserve': 'cx20fe6ac9331288dc1f05fee876b663da1862c980', 'daofund': 'cxd977af0b45d01b1343b6e2335a694d6c7ffda794', 'rewards': 'cxed28d820def132a5bdc3a9fa132965

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab', 'indexed': ['AssetAdded(Address,str,bool)', 'cx5e46cd5013a8f54a9b0d307d2995906cea3a5a2b', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab', 'indexed': ['AssetAdded(Address,str,bool)', 'cxaa47d8fda4ba82400984e6977be23963b5e7f4cc', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab', 'indexed': ['AssetAd

In [ ]:
# Cell 8
# Deploy or Update a single SCORE

compress()
update = 1
contract = contracts['loans']
params = {}
# params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)

In [14]:
# 1.no bad debt for asset, retired from account user2 without a position on Balanced.
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}


{'message': 'That address has no outstanding loans or deposited collateral.'}

In [22]:
call_tx('loans', 'getMaxRetireAmount', {'_symbol': 'bnUSD'})


------------------------------------------------------------------------------------------------------------------
Reading getMaxRetireAmount, with parameters {'_symbol': 'bnUSD'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
0x4562b068af5f0800


'0x4562b068af5f0800'

In [17]:
send_tx('loans', 2000*ICX, 'addCollateral', {'_asset': 'bnUSD', '_amount': 500 * ICX}, user1)


------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': 'bnUSD', '_amount': 500000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab', 'cx96eba15c7a6814d2a10235b01e0b89845d7cb4bb', '0x6c6b935b8bbd400000'], 'data': []} 

{'scoreAddress': 'cx5e46cd5013a8f54a9b0d307d2995906cea3a5a2b', 'indexed': ['Mint(Address,int,bytes)', 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab'], 'data': ['0x6c6b935b8bbd400000', '0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687865363264313166613139613065383537356164393266303662633866643432656462666532376462222c225f6173736574223a22626e555344222c225f616d6f756e74223a3530303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cx5e46cd5013a8f54a9b0d307d2995906cea3a5a2b', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000

{'txHash': '0xd028c3e206018263dfc6075ac7d6fd11c5597f244ca1378dd5431d37666295c7',
 'blockHeight': 2567509,
 'blockHash': '0xb29ed8b6364f1dba29c332602f02fa158a82b0e607771b18d92bd19e69c49791',
 'txIndex': 1,
 'to': 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab',
 'stepUsed': 1299700,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1299700,
 'eventLogs': [{'scoreAddress': 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab',
    'cx96eba15c7a6814d2a10235b01e0b89845d7cb4bb',
    '0x6c6b935b8bbd400000'],
   'data': []},
  {'scoreAddress': 'cx5e46cd5013a8f54a9b0d307d2995906cea3a5a2b',
   'indexed': ['Mint(Address,int,bytes)',
    'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab'],
   'data': ['0x6c6b935b8bbd400000',
    '0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a2268786536326431316661313961306538353735616439326630366263386664343265646266653237

In [21]:
#  Now we try to retire maximum bnusd retirement allowed  


test_cases =  { 
    
    "stories": [ 
        {
            "description": "User2 tries to retire 10 bnusd",
              "actions": {
                  "sender":"user2",
                  "first_meth": "transfer",
                  "second_meth":"transfer",
                  "deposited_icx":"0",
                  "first_params":{'_to': user2.get_address(), '_value': call_tx('loans', 'getMaxRetireAmount', {'_symbol': 'bnUSD'})},
                  "second_params":{"method": "_retire_asset", "params": {},'_to': contracts['loans']['SCORE'], '_value': call_tx('loans', 'getMaxRetireAmount', {'_symbol': 'bnUSD'})},
                  "expected_status_result":"1"
          }
        }
  ]
}

for case in test_cases['stories']:
    print(case['description'])
    _to = contracts['bnUSD']['SCORE']
    meth1 = case['actions']['first_meth']
    meth2 = case['actions']['second_meth']
    val = int(case['actions']['deposited_icx'])
    data1 = case['actions']['first_params']
    first_params = {"_to": data1['_to'], "_value": data1['_value']}
    
    data2 = case['actions']['second_params']
    params = {"method": data2['method'], "params": data2['params']}
    data = json.dumps(params).encode("utf-8")
    second_params = {'_to': data2['_to'], '_value': data2['_value'], '_data': data}

    send_tx('bnUSD', 0 ,meth1 , first_params, user1)
    sleep(2)
    res = send_tx('bnUSD', 0 ,meth2 , second_params, user2)
    assert res['status'] == int(case['actions']['expected_status_result']) , 'Retired amount is greater than the current maximum allowed'
    print('Test case passed')


------------------------------------------------------------------------------------------------------------------
Reading getMaxRetireAmount, with parameters {'_symbol': 'bnUSD'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
0x4615343e73b90000
------------------------------------------------------------------------------------------------------------------
Reading getMaxRetireAmount, with parameters {'_symbol': 'bnUSD'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
0x4615343e73b90000
User2 tries to retire 10 bnusd
------------------------------------------------------------------------------------------------------------------
Calling transfer, with parameters {'_to': 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '_value': '0x4615343e73b90000'} on the bnUSD contract.
-------------------------------------

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxaa47d8fda4ba82400984e6977be23963b5e7f4cc', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab', '0x4615343e73b90000'], 'data': ['0x7b226d6574686f64223a20225f7265746972655f6173736574222c2022706172616d73223a207b7d7d']} 

{'scoreAddress': 'cxaa47d8fda4ba82400984e6977be23963b5e7f4cc', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cxaa47d8fda4ba82400984e6977be23963b5e7f4cc', 'indexed': ['Burn(Address,int)', 'cxf73c31c4663c4e0e6df081e7966a77000a7f37ab'], 'data': ['0x45bb7f80b324e000']} 

{'scoreAddress': 'cxaa47d8fda4ba82400984e6977be23963b5e7f4cc', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxf73c31c4663c4e0e6df081e796